In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
#!pip install unsloth
!pip install vllm

In [11]:
import gc

torch.cuda.empty_cache()
gc.collect()

0

In [2]:
import pandas as pd

In [3]:
import torch
print(torch.__version__)

2.5.1+cu121


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel

#tokenizer = AutoTokenizer.from_pretrained("selfrag/selfrag_llama2_7b")
#model = AutoModelForCausalLM.from_pretrained("selfrag/selfrag_llama2_7b")
model = 'selfrag/selfrag_llama2_7b'
optimized_model, opt_tokenizer = FastLanguageModel.from_pretrained(model, max_seq_length = 256, load_in_4bit=True)

In [ ]:
FastLanguageModel.for_inference(optimized_model)

In [4]:
from datasets import load_dataset

dataset = load_dataset("philschmid/finanical-rag-embedding-dataset")

README.md:   0%|          | 0.00/882 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [ ]:
sample = dataset['train'][0]
question = sample['question']
context = sample['context']

def format_prompt(input, paragraph=None):
    prompt = "### Instruction:\n{0}\n\n### Response:\n".format(input)
    if paragraph is not None:
        prompt += "[Retrieval]<paragraph>{0}</paragraph>".format(paragraph)
    return prompt
formatted_prompt = format_prompt(input=question, paragraph=context)

# Tokenize the input
inputs = opt_tokenizer(formatted_prompt, return_tensors="pt", truncation=True, padding=True)


outputs = vllm_model.generate(
    inputs["input_ids"],
    max_length=500,  # Adjust as needed
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    #mode="adaptive_retrieval",  # Set adaptive retrieval mode
    #threshold=0.2,             # Threshold for retrieval
    #ndocs=5                    # Number of documents to retrieve
)

# Decode the generated output
generated_text = opt_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 5: Print the Result
print("Sample Input:")
print(f"Formatted Prompt:\n{formatted_prompt}")
print("\nGenerated Answer:")
print(generated_text)

In [12]:
# Load the VLLM library
from vllm import LLM, SamplingParams

# Step 1: Initialize the VLLM Model
# Specify the download directory and use half-precision for efficiency
model = LLM("selfrag/selfrag_llama2_7b", 
            download_dir="/gscratch/h2lab/akari/model_cache",
            dtype="half",
            cpu_offload_gb=4)

WARNING 12-13 23:17:18 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 12-13 23:17:18 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-13 23:17:18 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='selfrag/selfrag_llama2_7b', speculative_config=None, tokenizer='selfrag/selfrag_llama2_7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir='/gscratch/h2lab/akari/model_cache', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 62.12 MiB is free. Process 2624 has 14.65 GiB memory in use. Of the allocated memory 14.26 GiB is allocated by PyTorch, with 424.00 MiB allocated in private pools (e.g., CUDA Graphs), and 80.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
# Define sampling parameters
sampling_params = SamplingParams(
    temperature=0.0,   # Adjust randomness
    top_p=1.0,         # Nucleus sampling
    max_tokens=256,    # Maximum length of generated response
    skip_special_tokens=False  # Exclude special tokens in output
)

In [31]:
# Step 3: Prepare the Input
# Select a sample question and context
sample = dataset['train'][0]
question = sample['question']
context = sample['context']

# Define the format_prompt function
def format_prompt(input, paragraph=None):
    prompt = "### Instruction:\n{0}\n\n### Response:\n".format(input)
    if paragraph is not None:
        prompt += "[Retrieval]<paragraph>{0}</paragraph>".format(paragraph)
    
    return prompt

# Format the prompt
formatted_prompt = format_prompt(input=question, paragraph=context)

# Step 4: Run Inference
# Generate predictions using VLLM
preds = model.generate([formatted_prompt], sampling_params)

# Decode the generated response
generated_text = preds[0].outputs[0].text

# Step 5: Print the Result
print("Sample Input:")
print(f"Formatted Prompt:\n{formatted_prompt}")
print("\nGenerated Answer:")
print(generated_text)


Processed prompts: 100%|██████████| 1/1 [00:11<00:00, 11.05s/it, est. speed input: 5.34 toks/s, output: 2.35 toks/s]

Sample Input:
Formatted Prompt:
### Instruction:
What area did NVIDIA initially focus on before expanding to other computationally intensive fields?

### Response:
[Retrieval]<paragraph>Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.</paragraph>

Generated Answer:
[Relevant]NVIDIA initially focused on the field of PC graphics before expanding to other computationally intensive fields.[Fully supported][Utility:5]


In [32]:
query_1 = "Leave odd one out: twitter, instagram, whatsapp."
query_2 = "Can you tell me the difference between llamas and alpacas?"
queries = [query_1, query_2]

# for a query that doesn't require retrieval
preds = model.generate([format_prompt(query) for query in queries], sampling_params)
for pred in preds:
  print("Model prediction: {0}".format(pred.outputs[0].text))


Processed prompts: 100%|██████████| 2/2 [01:03<00:00, 31.61s/it, est. speed input: 0.89 toks/s, output: 3.39 toks/s]

Model prediction: Twitter, Instagram, and WhatsApp are all social media platforms.[No Retrieval]However, WhatsApp is a messaging app, while Twitter and Instagram are both primarily used for sharing photos and videos.[No Retrieval]Therefore, WhatsApp is the odd one out in this group.[Utility:5]
Model prediction: Sure![Retrieval]<paragraph>

* Alpaca (left) and llama (right) in the Andes of southern Peru.

Alpacas and llamas are both domesticated species of South American camelids.[Continue to Use Evidence]Alpacas are a much smaller than llamas, with a shoulder height of 3 to 4 feet.[Continue to Use Evidence]They are also bred specifically for their fiber, which is used to make all sorts of textiles and clothing.[Continue to Use Evidence]Llamas, on the other hand, are larger and more social, and are often kept as pets or as guard animals.[Continue to Use Evidence]They are also used as pack animals, and can carry up to 20% of their body weight.[Utility:5]


In [33]:
# for a query that needs factual grounding
prompt = format_prompt("Can you tell me the difference between llamas and alpacas?", "The alpaca (Lama pacos) is a species of South American camelid mammal. It is similar to, and often confused with, the llama. Alpacas are considerably smaller than llamas, and unlike llamas, they were not bred to be working animals, but were bred specifically for their fiber.")
preds = model.generate([prompt], sampling_params)
print([pred.outputs[0].text for pred in preds])
# ['[Relevant]Alpacas are considerably smaller than llamas, and unlike llamas, they were not bred to be working animals, but were bred specifically for their fiber.[Fully supported][Utility:5]</s>']


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.98s/it, est. speed input: 15.04 toks/s, output: 2.29 toks/s]

['[Relevant]Alpacas are considerably smaller than llamas.[Fully supported][Utility:5]']


In [36]:
prompt = format_prompt("What is the current trend of bitcoin price?", "Bitcoin is on an uptrend in the recent days. It recently crossed a major landmark of $100K and is now trading at 1000102.")
preds = model.generate([prompt], sampling_params)
print([pred.outputs[0].text for pred in preds])


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.28s/it, est. speed input: 5.70 toks/s, output: 2.36 toks/s]

['[Relevant]The current trend of Bitcoin price is upward, with the price increasing steadily over the past few months.[Partially supported][Utility:5]']
